In [4]:
import soccerdata as sd
import pandas as pd
import numpy as np
from datetime import datetime
import os
from pathlib import Path

# --- CONFIGURAÇÕES ---
SEASON = "2526"
LEAGUE = "ENG-Premier League"
CURRENT_DATE = datetime.now()

# Mostra todas as colunas no terminal
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

def extract_data():
    print(f"🔄 [1/3] Conectando ao FBref ({SEASON})...")
    cache_path = Path("./data/cache")
    fbref = sd.FBref(leagues=LEAGUE, seasons=SEASON, data_dir=cache_path)
    
    print("⬇️  [2/3] Baixando estatísticas de Goleiros (Match Logs)...")
    # A tabela 'keepers' nos match logs já contém PSxG, Crosses e Sweeper actions
    keepers = fbref.read_player_match_stats(stat_type="keepers")
    
    return keepers

def clean_and_process(df):
    print("⚙️  [3/3] Limpando e organizando...")
    df = df.reset_index()
    
    # PADRONIZAÇÃO DE COLUNAS (O Pulo do Gato)
    # O soccerdata devolve colunas MultiIndex (ex: ('Shot Stopping', 'PSxG')).
    # Vamos achatar para nomes simples como 'Shot_Stopping_PSxG'.
    new_cols = []
    for col in df.columns:
        if isinstance(col, tuple):
            # Junta os níveis com underscore e remove espaços
            new_name = f"{col[0]}_{col[1]}".strip().replace(" ", "_")
            # Se o primeiro nível for vazio ou inútil, pega só o segundo
            if col[0] in ['', 'Unnamed: 0_level_0']: 
                new_name = col[1]
            new_cols.append(new_name)
        else:
            new_cols.append(col)
    
    df.columns = new_cols

    # Filtro de data (apenas jogos passados)
    if 'date' in df.columns:
        df['date'] = pd.to_datetime(df['date'])
        df = df[(df['date'] <= CURRENT_DATE) & (df['minutes'] > 0)].copy()
    
    # Renomear para nomes amigáveis para nosso projeto
    # Ajuste aqui baseado nos nomes que o 'achatar' gerou
    cols_map = {
        'date': 'Date',
        'player': 'Player',
        'team': 'Team',
        'opponent': 'Opponent',
        'Shot_Stopping_GA': 'GA',
        'Shot_Stopping_Saves': 'Saves',
        'Shot_Stopping_PSxG': 'PSxG',
        'Crosses_Opp': 'Crosses_Faced',
        'Crosses_Stp': 'Crosses_Stopped',
        'Sweeper_#OPA': 'OPA',
        'Sweeper_AvgDist': 'Sweeper_Dist'
    }
    
    # Renomeia apenas o que encontrar
    found_cols = {k: v for k, v in cols_map.items() if k in df.columns}
    df = df.rename(columns=found_cols)
    
    # Tratamento de Nulos
    cols_to_fill = ['PSxG', 'GA', 'Crosses_Faced', 'Crosses_Stopped', 'OPA']
    for c in cols_to_fill:
        if c in df.columns:
            df[c] = df[c].fillna(0)
            
    # Criação da Métrica Principal (PSxG +/-)
    if 'PSxG' in df.columns and 'GA' in df.columns:
        df['PSxG_Net'] = df['PSxG'] - df['GA']
        
    return df

if __name__ == "__main__":
    os.makedirs("data/processed", exist_ok=True)
    
    try:
        raw_data = extract_data()
        df_final = clean_and_process(raw_data)
        
        output_path = "data/processed/pl_goalkeepers_2526_matches.csv"
        df_final.to_csv(output_path, index=False)
        
        print(f"\n✅ Sucesso! Arquivo salvo: {output_path}")
        print(f"📊 Registros: {len(df_final)}")
        print("Colunas disponíveis:", df_final.columns.tolist())
        
    except Exception as e:
        print(f"\n❌ Erro: {e}")

🔄 [1/3] Conectando ao FBref (2526)...


[01/10/26 19:53:42] INFO     Saving cached data to data\cache                                        ]8;id=930441;file://c:\Users\jvamg\Documents\fds\powerranking\.venv\Lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=551177;file://c:\Users\jvamg\Documents\fds\powerranking\.venv\Lib\site-packages\soccerdata\_common.py#263\263]8;;\

⬇️  [2/3] Baixando estatísticas de Goleiros (Match Logs)...


                    WARNING  c:\Users\jvamg\Documents\fds\powerranking\.venv\Lib\site-packages\socc warnings.py:110
                             erdata\fbref.py:165: FutureWarning: The behavior of DataFrame                         
                             concatenation with empty or all-NA entries is deprecated. In a future                 
                             version, this will no longer exclude empty or all-NA columns when                     
                             determining the result dtypes. To retain the old behavior, exclude the                
                             relevant entries before the concat operation.                                         
                               pd.concat(dfs)                                                                      
                                                                                                                   

[01/10/26 19:53:49] INFO     [1/210] Retrieving game with id=a071faa8                                  ]8;id=322612;file://c:\Users\jvamg\Documents\fds\powerranking\.venv\Lib\site-packages\soccerdata\fbref.py\fbref.py]8;;\:]8;id=469059;file://c:\Users\jvamg\Documents\fds\powerranking\.venv\Lib\site-packages\soccerdata\fbref.py#815\815]8;;\

...

[01/10/26 19:54:49] INFO     [207/210] Retrieving game with id=21522578                                ]8;id=768600;file://c:\Users\jvamg\Documents\fds\powerranking\.venv\Lib\site-packages\soccerdata\fbref.py\fbref.py]8;;\:]8;id=648374;file://c:\Users\jvamg\Documents\fds\powerranking\.venv\Lib\site-packages\soccerdata\fbref.py#815\815]8;;\

                    INFO     [208/210] Retrieving game with id=009bbc59                                ]8;id=746055;file://c:\Users\jvamg\Documents\fds\powerranking\.venv\Lib\site-packages\soccerdata\fbref.py\fbref.py]8;;\:]8;id=821490;file://c:\Users\jvamg\Documents\fds\powerranking\.venv\Lib\site-packages\soccerdata\fbref.py#815\815]8;;\

                    INFO     [209/210] Retrieving game with id=460a6353                                ]8;id=10751;file://c:\Users\jvamg\Documents\fds\powerranking\.venv\Lib\site-packages\soccerdata\fbref.py\fbref.py]8;;\:]8;id=983466;file://c:\Users\jvamg\Documents\fds\powerranking\.venv\Lib\site-packages\soccerdata\fbref.py#815\815]8;;\

[01/10/26 19:54:50] INFO     [210/210] Retrieving game with id=2b794de5                                ]8;id=491155;file://c:\Users\jvamg\Documents\fds\powerranking\.venv\Lib\site-packages\soccerdata\fbref.py\fbref.py]8;;\:]8;id=955826;file://c:\Users\jvamg\Documents\fds\powerranking\.venv\Lib\site-packages\soccerdata\fbref.py#815\815]8;;\

⚙️  [3/3] Limpando e organizando...

✅ Sucesso! Arquivo salvo: data/processed/pl_goalkeepers_2526_matches.csv
📊 Registros: 424
Colunas disponíveis: ['league_', 'season_', 'game_', 'team_', 'player_', 'nation_', 'age_', 'min_', 'Shot_Stopping_SoTA', 'GA', 'Saves', 'Shot_Stopping_Save%', 'PSxG', 'Launched_Cmp', 'Launched_Att', 'Launched_Cmp%', 'Passes_Att_(GK)', 'Passes_Thr', 'Passes_Launch%', 'Passes_AvgLen', 'Goal_Kicks_Att', 'Goal_Kicks_Launch%', 'Goal_Kicks_AvgLen', 'Crosses_Faced', 'Crosses_Stopped', 'Crosses_Stp%', 'OPA', 'Sweeper_Dist', 'game_id_', 'PSxG_Net']
